In [1]:
import numpy as np
import pyvista as pv
# import project_heart as ph
from project_heart.modules.geometry import Geometry
pv.set_jupyter_backend("pythreejs")

In [13]:
# jupyter_backend='pythreejs'
pv.set_jupyter_backend("pythreejs")

In [ ]:
FILE_PATH = "C:/Users/igorp/University of South Florida/Wenbin Mao - Igor/LV_Meshes/Heart_models/Full_Heart_Mesh_1.vtk"

In [5]:
lv = Geometry()
lv.from_pyvista_read(FILE_PATH, identifier="elemTag", threshold=[0, 1])

NameError: name 'Geometry' is not defined

In [ ]:
lvsurf = lv.mesh.extract_surface()
pts = np.array(lvsurf.points).astype(np.float32)


In [ ]:
from sklearn.cluster import KMeans

n_centroids = 2

kmeans = KMeans(n_clusters=n_centroids, random_state=0).fit(pts)
label = kmeans.labels_
kcenters = kmeans.cluster_centers_
kcenters = kcenters[np.argsort(kcenters[:,-1])]

In [ ]:
clustered = np.zeros(len(pts))
clustered = label 
lvsurf.point_data["clustered"] = clustered
lvsurf.set_active_scalars("clustered")

plotter = pv.Plotter()
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
# plotter.add_mesh(lvsurf.arrows, lighting=False, scalars="angles")
plotter.add_points(kcenters, color="red", point_size=300)
plotter.add_mesh(lvsurf, scalars="clustered", opacity=1.0, show_edges=False)
plotter.show()

In [ ]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

In [ ]:
from scipy.spatial.transform import Rotation as tr
def get_rotation(from_vec, to_vector):
  # USING APPROACH FROM:
  # https://bit.ly/2W9gNb5

  # copy vectors so we dont modify them directly
  from_vec = np.copy(from_vec)
  to_vector = np.copy(to_vector)

  # Make unit vector
  to_vector = unit_vector(to_vector)
  from_vec = unit_vector(from_vec)

  v = np.cross(from_vec, to_vector) # cross product
  s = np.abs(v) # sine of angle
  c = np.dot(from_vec, to_vector) # cosine of angle
  # get skew-symmetric cross-product matrix of v
  vx = np.array([
      [0, -v[2], v[1]],
      [v[2], 0, -v[0]],
      [-v[1], v[0], 0]
  ])
  # compute rotation matrix
  rot_matrix = np.identity(3) + vx + vx**2 * (1/(1+c))

  # create rotation object from rotation matrix
  rot = tr.from_matrix(rot_matrix)
  return rot

In [ ]:
from collections import deque
rot_chain = deque()

In [ ]:
# set vectors for rotation
znormal = np.array([0.,0.,1.])
edge_long_vec = kcenters[1] - kcenters[0]

# get rotation matrix
rot = get_rotation(edge_long_vec, znormal)
rot_chain.append(rot)

# apply rotation (don't modify the results directly, 
# so we can display them)
lvsurfpts = np.array(lvsurf.points)
lvsurfpts_after_rot_1 = rot.apply(lvsurfpts)

In [ ]:
def get_apex_ref(points, ql=0.03, **kwargs):
  zvalues = points[:, 2]
  thresh = np.quantile(zvalues, ql)
  apex_region_idxs = np.where(zvalues <= thresh)[0]
  apex_region_pts = points[apex_region_idxs]
  return np.mean(apex_region_pts, 0), apex_region_idxs

In [ ]:
def get_base_ref(points, qh=0.90, **kwargs):
  zvalues = points[:, 2]
  thresh = np.quantile(zvalues, qh)
  base_region_idxs = np.where(zvalues >= thresh)[0]
  base_region_pts = points[base_region_idxs]
  base_ref = np.mean(base_region_pts, 0)
  return base_ref, base_region_idxs

In [ ]:
def get_refs(points, **kwargs):
  apex_ref, apex_region_idxs = get_apex_ref(points, **kwargs)
  base_ref, base_region_idxs = get_base_ref(points, **kwargs)
  extra = {"apex_region": apex_region_idxs, 
           "base_region": base_region_idxs}
  return np.vstack((base_ref, apex_ref)), extra

In [ ]:
def angle_between(v1, v2, zaxis=[0.,0.,1.]):
    """ 
      Returns the angle in radians between vectors 'v1' and 'v2'
    """
    #  compute angle
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    angle = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    # make sure angle is in range [0, 2*pi)
    zaxis = np.asarray(zaxis)
    det = np.linalg.det(np.vstack((v1_u.T, v2_u.T, zaxis.T))) # https://bit.ly/3nUrr0U
    if det < 0:
      angle = 2*np.pi - angle
    return angle

In [ ]:
def align_with_normal(points, n=5, rot_chain=[], **kwargs):
  pts = np.copy(points)
  for _ in range(n):
    long_line, _ = get_refs(pts, **kwargs)
    lv_normal = unit_vector(long_line[0] - long_line[1])
    print(np.degrees(angle_between(lv_normal, znormal)))
    curr_rot = get_rotation(lv_normal, znormal)
    pts = curr_rot.apply(pts)
    rot_chain.append(curr_rot)
  long_line, extra = get_refs(pts, **kwargs)
  lv_normal = unit_vector(long_line[0] - long_line[1])
  extra["normal"] = lv_normal
  extra["long_line"] = long_line
  extra["rot_chain"] = rot_chain
  print(np.degrees(angle_between(lv_normal, znormal)))
  return pts, extra

In [ ]:
org_longline, org_info = get_refs(lvsurfpts_after_rot_1, ql=0.1, qh=0.6) # for plot
lvsurfpts_after_rot_2, adj_info = align_with_normal(lvsurfpts_after_rot_1, 
                                                    rot_chain=rot_chain, 
                                                    n=10, ql=0.03, qh=0.75)
long_line = adj_info["long_line"]
lv_normal = adj_info["normal"]

In [ ]:
adj_info["apex_region"]

In [ ]:
clustered = np.zeros(len(pts))
clustered[adj_info["apex_region"]] = 1 
clustered[adj_info["base_region"]] = 2

lvsurf.point_data["clustered"] = clustered
lvsurf.set_active_scalars("clustered")

plotter = pv.Plotter()
plotter.background_color = 'w'
plotter.enable_anti_aliasing()
# plotter.add_mesh(lvsurf.arrows, lighting=False, scalars="angles")
plotter.add_points(kcenters, color="red", point_size=300)
plotter.add_mesh(lvsurf, scalars="clustered", opacity=1.0, show_edges=False)
plotter.show()